In [3]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [9]:
# 1. URL 파라미터 분리하기.
# Service URL
xmlUrl = 'http://api.odcloud.kr/api/15077756/v1/vaccine-stat?page=1&perPage=10'

'http://api.odcloud.kr/api/15077756/v1/vaccine-stat?page=1&perPage=10'

In [8]:
My_API_Key = unquote('%2BiCZUbrZgFKELEU40o3o%2BhZ%2F98%2Bjo54c8P%2FnQWi4GoHi%2FdTVIfanE7vPqK16POCXWEwfJ6%2ByKIGBLD8nFzMR6A%3D%3D')    # 아래 내가 받은 인증키가 안 되서 수업용 인증키 사용.

queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('ServiceKey') : My_API_Key,         # 필수 항목 1 : 서비스키 (본인의 서비스키)
        quote_plus('startCreateDt') : '20210310',      # 필수 항목 2 : 지역코드 (법정코드목록조회에서 확인)
        quote_plus('endCreateDt') : '20210331'         # 픽수 항목 3 : 계약월
     }
)


SyntaxError: invalid syntax (<ipython-input-8-80ba940545b7>, line 1)

In [6]:
response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
xmlobj

<?xml version="1.0" encoding="utf-8"?>

In [7]:
rows = xmlobj.findAll('item')
rows[0]

IndexError: list index out of range

In [39]:
columns = rows[0].find_all()
columns

[<createDt>2021-03-31 09:36:15.516</createDt>,
 <deathCnt>4</deathCnt>,
 <defCnt>3158</defCnt>,
 <gubun>검역</gubun>,
 <gubunCn>隔離區</gubunCn>,
 <gubunEn>Lazaretto</gubunEn>,
 <incDec>7</incDec>,
 <isolClearCnt>2824</isolClearCnt>,
 <isolIngCnt>330</isolIngCnt>,
 <localOccCnt>0</localOccCnt>,
 <overFlowCnt>7</overFlowCnt>,
 <qurRate>-</qurRate>,
 <seq>8614</seq>,
 <stdDay>2021년 03월 31일 00시</stdDay>,
 <updateDt>null</updateDt>]

In [41]:
# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)
result

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
0,2021-03-31 09:36:15.516,4,3158,검역,隔離區,Lazaretto,7,2824,330,0,7,-,8614,2021년 03월 31일 00시,null
1,2021-03-31 09:36:15.515,1,625,제주,济州,Jeju,2,613,11,2,0,93.18,8613,2021년 03월 31일 00시,null
2,2021-03-31 09:36:15.515,15,2960,경남,庆南,Gyeongsangnam-do,22,2618,327,21,1,88.06,8612,2021년 03월 31일 00시,null
3,2021-03-31 09:36:15.515,75,3504,경북,庆北,Gyeongsangbuk-do,6,3289,140,6,0,131.61,8611,2021년 03월 31일 00시,null
4,2021-03-31 09:36:15.515,8,913,전남,全南,Jeollanam-do,0,882,23,0,0,48.96,8610,2021년 03월 31일 00시,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7360,2020-03-10 10:58:20.20,0,13,인천,NULL,NULL,4,3,0.44,132,2020년 3월 10일 00시,NULL,None,None,None
7361,2020-03-10 10:58:20.20,38,5663,대구,NULL,NULL,92,92,232.42,131,2020년 3월 10일 00시,NULL,None,None,None
7362,2020-03-10 10:58:20.20,0,96,부산,NULL,NULL,0,10,2.81,130,2020년 3월 10일 00시,NULL,None,None,None
7363,2020-03-10 10:58:20.20,0,141,서울,NULL,NULL,11,31,1.45,129,2020년 3월 10일 00시,NULL,None,None,None


In [42]:
result.to_csv('3.코로나19 시도발생_현황.csv')